In [1]:
import json
import pandas as pd
from pathlib import Path

In [2]:
# Generalised demeaning helper function
# Declaring helper demeaning function to re-use
def demean_2FE(df):
    # Copying dataframe
    df_fe = df.copy()
    # Looping over all columns
    for col in df.columns:
        # Grouping by FE type and and create mean vector
        e_mean = df_fe[col].groupby(level='entity_id').transform('mean')
        t_mean = df_fe[col].groupby(level='year').transform('mean')
        total_mean = df_fe[col].mean()
        # Demeaning
        df_fe[col] = df_fe[col] - e_mean - t_mean + total_mean
    return df_fe

In [3]:
IN = Path("data/processed")

final = json.loads((IN / "final.json").read_text())
df = pd.read_parquet(IN / "bfs_data.parquet")

y = final["y"]
Z = final["Z_selected"]

# Rebuilding multi-index
dfp = df.set_index([final["entity_level"], final["time_level"]]).sort_index()

# Demeaning using helper function
df_dm = demean_2FE(dfp[[y] + Z].dropna())
y_dm = df_dm[y].to_numpy()
X_dm = df_dm[Z].to_numpy()